# Regressione lineare con Iris (da CSV)

Notebook per una lezione di ~4 ore (studenti junior)


## Obiettivi della lezione
- Capire l’idea di **regressione**: prevedere un numero (variabile continua)
- Caricare e ispezionare un dataset da `iris.csv`
- Costruire un modello di **regressione lineare** con scikit-learn
- Valutare il modello con metriche e grafici (errori, residui)
- Estendere a **regressione multipla** e, opzionale, **polinomiale**

## Prerequisiti
- Python installato
- VS Code + estensione Jupyter
- File `iris.csv` nella stessa cartella del notebook (oppure aggiorna il path)

## Agenda (indicativa)
1. Caricamento dati e controlli (30–40 min)
2. Esplorazione dati con grafici (50–60 min)
3. Regressione lineare semplice: train/test, metriche, grafici (60–70 min)
4. Regressione multipla: interpretazione coefficienti, confronto (40–50 min)
5. Estensioni e mini-esercizi (30–40 min)


## 1) Setup e import
Esegui questa cella per importare librerie e impostare alcune opzioni grafiche.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Per rendere i grafici più leggibili
plt.rcParams["figure.figsize"] = (9, 5)
plt.rcParams["axes.grid"] = True

print("Setup OK")

## 2) Carica `iris.csv`

Metti `iris.csv` **nella stessa cartella** di questo notebook.
Se il file è altrove, modifica `csv_path`.

Nota: Iris nasce come dataset di **classificazione** (specie). Qui lo usiamo per una **regressione**: prevediamo una misura (es. `petal_length`) da altre misure.

In [ ]:
csv_path = "iris.csv"  # cambia qui se necessario
df = pd.read_csv(csv_path)

df.head()

### Controllo rapido: colonne, tipi, valori mancanti

In [ ]:
display(df.info())
display(df.describe(include="all"))

missing = df.isna().sum().sort_values(ascending=False)
missing[missing > 0]

## 3) Pulizia minima e standardizzazione nomi colonne

Dataset Iris da CSV spesso ha nomi tipo `sepal_length`, oppure `SepalLengthCm`. Normalizziamo i nomi e facciamo una scelta chiara.

Se il tuo CSV ha già nomi puliti, questa cella non fa danni (rinomina solo quando serve).

In [ ]:
# Normalizza nomi: minuscolo, spazi->underscore
df.columns = [c.strip().lower().replace(" ", "_") for c in df.columns]

# Rinomine comuni (se presenti)
rename_map = {
    "sepallengthcm": "sepal_length",
    "sepalwidthcm": "sepal_width",
    "petallengthcm": "petal_length",
    "petalwidthcm": "petal_width",
    "species": "species"
}
df = df.rename(columns={c: rename_map[c] for c in df.columns if c in rename_map})

df.columns

## 4) Esplorazione dati (EDA) con grafici

Qui facciamo grafici “belli” e immediati.

### 4.1 Distribuzioni (istogrammi)
Scegliamo le colonne numeriche e vediamo come sono distribuite.

In [ ]:
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
num_cols

In [ ]:
df[num_cols].hist(bins=20)
plt.suptitle("Distribuzioni delle variabili numeriche")
plt.show()

### 4.2 Scatter plot: relazione tra due variabili

Per la regressione lineare semplice useremo:
- **X = sepal_length**
- **y = petal_length**

Motivo: c’è spesso una relazione abbastanza lineare e si vede bene a occhio.

In [ ]:
x_col = "sepal_length"
y_col = "petal_length"

if x_col not in df.columns or y_col not in df.columns:
    raise ValueError(f"Colonne attese non trovate. Colonne disponibili: {list(df.columns)}")

plt.scatter(df[x_col], df[y_col], alpha=0.7)
plt.xlabel(x_col)
plt.ylabel(y_col)
plt.title(f"Scatter: {y_col} vs {x_col}")
plt.show()

### 4.3 Correlazioni (heatmap semplice)

Non usiamo librerie extra: solo matplotlib.

In [ ]:
corr = df[num_cols].corr(numeric_only=True)

plt.imshow(corr.values)
plt.xticks(range(len(corr.columns)), corr.columns, rotation=45, ha="right")
plt.yticks(range(len(corr.columns)), corr.columns)
plt.colorbar()
plt.title("Matrice di correlazione")
plt.tight_layout()
plt.show()

corr

## 5) Regressione lineare semplice (una X → una y)

### Idea in 20 secondi
Vogliamo una retta:

\[ \hat{y} = a \cdot x + b \]

che “passi vicino” ai punti.

### Definiamo X e y

In [ ]:
X = df[[x_col]].copy()   # DataFrame 2D
y = df[y_col].copy()     # Series 1D

X.shape, y.shape

### 5.1 Train/Test split

Separiamo dati in:
- **train**: per allenare
- **test**: per valutare in modo onesto

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42
)

X_train.shape, X_test.shape

### 5.2 Allenamento del modello

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)

a = float(model.coef_[0])
b = float(model.intercept_)

print("Modello allenato")
print(f"y_hat = {a:.3f} * x + {b:.3f}")

### 5.3 Predizioni e metriche

Useremo 3 metriche:
- **MAE**: errore medio assoluto (facile da capire)
- **RMSE**: penalizza di più gli errori grandi
- **R²**: “quanto spiega” il modello (0–1, ma può essere anche negativo se pessimo)

In [ ]:
y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print(f"MAE  = {mae:.3f}")
print(f"RMSE = {rmse:.3f}")
print(f"R²   = {r2:.3f}")

### 5.4 Grafico: punti reali + retta prevista

Questo è il grafico più “didattico”: i punti e la retta.

In [ ]:
# Scatter dei dati
plt.scatter(X_test[x_col], y_test, label="Reale (test)", alpha=0.7)

# Linea della regressione: prendiamo un range di x e calcoliamo y_hat
x_line = np.linspace(df[x_col].min(), df[x_col].max(), 100)
y_line = model.predict(pd.DataFrame({x_col: x_line}))

plt.plot(x_line, y_line, label="Retta del modello")
plt.xlabel(x_col)
plt.ylabel(y_col)
plt.title("Regressione lineare semplice: test set + retta")
plt.legend()
plt.show()

### 5.5 Grafico: reale vs predetto

Se fosse perfetto, i punti starebbero sulla diagonale.

In [ ]:
plt.scatter(y_test, y_pred, alpha=0.7)
lims = [min(y_test.min(), y_pred.min()), max(y_test.max(), y_pred.max())]
plt.plot(lims, lims)
plt.xlabel("y reale (test)")
plt.ylabel("y predetta")
plt.title("Reale vs Predetto")
plt.show()

### 5.6 Residui (errori) e cosa ci dicono

Residuo = `y_reale - y_predetta`.

Un buon modello ha residui:
- centrati attorno a 0
- senza pattern evidenti quando li mettiamo contro `x`

In [ ]:
residuals = y_test - y_pred

plt.hist(residuals, bins=20)
plt.title("Distribuzione dei residui (test)")
plt.xlabel("Residuo")
plt.ylabel("Conteggio")
plt.show()

plt.scatter(X_test[x_col], residuals, alpha=0.7)
plt.axhline(0)
plt.title("Residui vs X (test)")
plt.xlabel(x_col)
plt.ylabel("Residuo")
plt.show()

## 6) Mini-esercizio (10–15 min)

1. Cambia `x_col` con un’altra colonna numerica (es. `sepal_width`).
2. Riesegui le celle dalla sezione 4.2 in poi.
3. Confronta MAE / RMSE / R² e i grafici.

Domanda: **qual è la migliore singola variabile per prevedere `petal_length`?**

Suggerimento: prova 2–3 opzioni, non tutte.

## 7) Regressione multipla (più X → una y)

Ora usiamo più colonne come input, ad esempio:
- `sepal_length`, `sepal_width`, `petal_width`

e teniamo `y = petal_length`.

In pratica: la “retta” diventa un **piano/iperpiano**.

In [ ]:
feature_cols = ["sepal_length", "sepal_width", "petal_width"]
for c in feature_cols + [y_col]:
    if c not in df.columns:
        raise ValueError(f"Manca la colonna {c}. Colonne disponibili: {list(df.columns)}")

X2 = df[feature_cols].copy()
y2 = df[y_col].copy()

X2_train, X2_test, y2_train, y2_test = train_test_split(
    X2, y2, test_size=0.25, random_state=42
)

model2 = LinearRegression()
model2.fit(X2_train, y2_train)

y2_pred = model2.predict(X2_test)

mae2 = mean_absolute_error(y2_test, y2_pred)
rmse2 = mean_squared_error(y2_test, y2_pred, squared=False)
r2_2 = r2_score(y2_test, y2_pred)

print("Metriche regressione multipla")
print(f"MAE  = {mae2:.3f}")
print(f"RMSE = {rmse2:.3f}")
print(f"R²   = {r2_2:.3f}")

### 7.1 Coefficienti: “quanto pesa” ogni variabile

Attenzione: senza normalizzare le feature, i coefficienti dipendono dalle scale.
Qui le scale sono simili, quindi è comunque leggibile per una lezione junior.

In [ ]:
coef_table = pd.DataFrame({
    "feature": feature_cols,
    "coef": model2.coef_
}).sort_values("coef", ascending=False)

display(coef_table)

plt.bar(coef_table["feature"], coef_table["coef"])
plt.title("Coefficienti della regressione multipla")
plt.xlabel("Feature")
plt.ylabel("Coefficiente")
plt.show()

### 7.2 Grafico: reale vs predetto (multipla)

In [ ]:
plt.scatter(y2_test, y2_pred, alpha=0.7)
lims = [min(y2_test.min(), y2_pred.min()), max(y2_test.max(), y2_pred.max())]
plt.plot(lims, lims)
plt.xlabel("y reale (test)")
plt.ylabel("y predetta")
plt.title("Reale vs Predetto (regressione multipla)")
plt.show()

## 8) Confronto semplice: modello 1 vs modello 2

Quale generalizza meglio?

Ricorda: un modello con più feature *può* migliorare, ma non è garantito.

In [ ]:
comparison = pd.DataFrame([
    {"modello": "Semplice (sepal_length -> petal_length)", "MAE": mae, "RMSE": rmse, "R2": r2},
    {"modello": "Multipla (3 feature -> petal_length)", "MAE": mae2, "RMSE": rmse2, "R2": r2_2},
])
comparison

## 9) Estensione opzionale: regressione polinomiale (sezione “wow”)

Se gli studenti sono molto junior, questa parte può essere solo dimostrativa.

Idea: aggiungiamo `x²` per permettere una curva.

Useremo **solo numpy** (senza pipeline) per restare leggibili.

In [ ]:
# Regressione polinomiale di grado 2: y_hat = a2*x^2 + a1*x + b
X_poly = pd.DataFrame({
    x_col: df[x_col],
    f"{x_col}^2": df[x_col] ** 2
})
y_poly = df[y_col]

Xp_train, Xp_test, yp_train, yp_test = train_test_split(
    X_poly, y_poly, test_size=0.25, random_state=42
)

modelp = LinearRegression()
modelp.fit(Xp_train, yp_train)
yp_pred = modelp.predict(Xp_test)

maep = mean_absolute_error(yp_test, yp_pred)
rmsep = mean_squared_error(yp_test, yp_pred, squared=False)
r2p = r2_score(yp_test, yp_pred)

print("Metriche polinomiale (grado 2)")
print(f"MAE  = {maep:.3f}")
print(f"RMSE = {rmsep:.3f}")
print(f"R²   = {r2p:.3f}")

In [ ]:
# Grafico: dati + curva (usiamo il modello polinomiale)
plt.scatter(df[x_col], df[y_col], alpha=0.5, label="Dati")

x_line = np.linspace(df[x_col].min(), df[x_col].max(), 200)
X_line_poly = pd.DataFrame({x_col: x_line, f"{x_col}^2": x_line**2})
y_line = modelp.predict(X_line_poly)

plt.plot(x_line, y_line, label="Curva polinomiale (grado 2)")
plt.xlabel(x_col)
plt.ylabel(y_col)
plt.title("Regressione polinomiale (dimostrazione)")
plt.legend()
plt.show()

## 10) Esercizi finali (30–40 min)

1. Cambia la variabile target `y_col` in `petal_width` e ripeti i modelli.
2. Prova un diverso split train/test (es. `test_size=0.3`) e osserva come cambiano le metriche.
3. (Bonus) Aggiungi una feature in `feature_cols` e osserva l’impatto.

Obiettivo: collegare **scelte** (feature, target, split) a **risultati** (metriche e grafici).

## 11) Riepilogo

- La regressione lineare prova a descrivere una relazione con una formula semplice.
- La valutazione va fatta su dati non visti (test set).
- Grafici e residui aiutano a capire se il modello “ha senso”.
- Più feature possono aiutare, ma vanno valutate.

Fine.